## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 svizor/zoomcamp-model:3.11.5-hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'has_subscribed': True, 'has_subscribed_probability': <value>}
```

Here `<value>` is the probability of getting a subscription. You need to choose the right one.

* 0.287
* 0.530
* 0.757   `<--`
* 0.960

## Use `kind` to create a local Kubernetes cluster

In [4]:
!kubectl --help

kubectl controls the Kubernetes cluster manager.

 Find more information at: https://kubernetes.io/docs/reference/kubectl/

Basic Commands (Beginner):
  create          Create a resource from a file or from stdin
  expose          Take a replication controller, service, deployment or pod and expose it as a new Kubernetes service
  run             Run a particular image on the cluster
  set             Set specific features on objects

Basic Commands (Intermediate):
  explain         Get documentation for a resource
  get             Display one or many resources
  edit            Edit a resource on the server
  delete          Delete resources by file names, stdin, resources and names, or by resources and label selector

Deploy Commands:
  rollout         Manage the rollout of a resource
  scale           Set a new size for a deployment, replica set, or replication controller
  autoscale       Auto-scale a deployment, replica set, stateful set, or replication controller

Cluster Manage

### Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.

In [5]:
!kind --version

kind version 0.25.0


In [10]:
!kind create cluster --name hw09

Creating cluster "hw09" ...
 • Ensuring node image (kindest/node:v1.31.2) 🖼  ...
 ✓ Ensuring node image (kindest/node:v1.31.2) 🖼
 • Preparing nodes 📦   ...
 ✓ Preparing nodes 📦 
 • Writing configuration 📜  ...
 ✓ Writing configuration 📜
 • Starting control-plane 🕹️  ...
 ✓ Starting control-plane 🕹️
 • Installing CNI 🔌  ...
 ✓ Installing CNI 🔌
 • Installing StorageClass 💾  ...
 ✓ Installing StorageClass 💾
Set kubectl context to "kind-hw09"
You can now use your cluster with:

kubectl cluster-info --context kind-hw09

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂


In [ ]:
!kubectl cluster-info --context kind-hw09

Kubernetes control plane is running at https://127.0.0.1:58171
CoreDNS is running at https://127.0.0.1:58171/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

* Node
* Pod  `<--`
* Deployment
* Service

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* NodePort
* ClusterIP   `<--`
* ExternalName
* LoadBalancer

In [1]:
!kubectl get service

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   77m


In [2]:
!kubectl get deployment

No resources found in default namespace.


In [3]:
!kubectl get pod

No resources found in default namespace.


In [4]:
!pip install GPUtil

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7399 sha256=d59a796e4ac235e84aad50e368b5f557079e0e0cb8116afabca9cbc4037c7e62
  Stored in directory: c:\users\julrich\appdata\local\pip\cache\wheels\2b\4d\8f\55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [5]:
import GPUtil

GPUtil.getAvailable()

[]

In [7]:
import os

os.cpu_count()


4

## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:3.11.5-hw10`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`   `<--`
* `kubectl apply`

In [11]:
!kind load docker-image --name hw09 svizor/zoomcamp-model:3.11.5-hw10

Image: "svizor/zoomcamp-model:3.11.5-hw10" with ID "sha256:020904a2523cec81c854a9b4679ae26d23f814e42350721136d1447f910a6b53" not yet present on node "hw09-control-plane", loading...


## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?   `9696`

Apply this deployment using the appropriate command and get a list of running Pods. 

In [19]:
!dir

 Volume in drive C is Windows 10 Pro
 Volume Serial Number is 6A14-E860

 Directory of c:\Projects\learning\datatalksclub\machine-learning-zoomcamp\cohorts\2024\10-kubernetes

13/12/2024  13:56    <DIR>          .
13/12/2024  13:56    <DIR>          ..
13/12/2024  13:56             6,453 homework.md
13/12/2024  15:56            11,626 hw10.ipynb
13/12/2024  14:27    <DIR>          kubernetes
               2 File(s)         18,079 bytes
               3 Dir(s)  40,377,585,664 bytes free


In [22]:
!kubectl apply -f kubernetes/deployment.yaml

deployment.apps/subscription created


In [23]:
!kubectl get deployment

NAME           READY   UP-TO-DATE   AVAILABLE   AGE
subscription   1/1     1            1           60s


In [24]:
!kubectl get pods

NAME                           READY   STATUS    RESTARTS   AGE
subscription-dd89dfd5f-qgf86   1/1     Running   0          86s


In [26]:
!kubectl describe pod subscription-dd89dfd5f-qgf86

Name:             subscription-dd89dfd5f-qgf86
Namespace:        default
Priority:         0
Service Account:  default
Node:             hw09-control-plane/172.18.0.2
Start Time:       Fri, 13 Dec 2024 16:08:05 +0000
Labels:           app=subscription
                  pod-template-hash=dd89dfd5f
Annotations:      <none>
Status:           Running
IP:               10.244.0.5
IPs:
  IP:           10.244.0.5
Controlled By:  ReplicaSet/subscription-dd89dfd5f
Containers:
  subscription:
    Container ID:   containerd://e1d13b956bf2714ae37db6af3bbb78959a2a9c51c1919f7eb0948a5e1b544831
    Image:          svizor/zoomcamp-model:3.11.5-hw10
    Image ID:       docker.io/library/import-2024-12-13@sha256:ed075d03a306c99aa765898c1bb76e7013cd6f46ed963e6c2bb219f648ed58c4
    Port:           9696/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Fri, 13 Dec 2024 16:08:06 +0000
    Ready:          True
    Restart Count:  0
    Limits:
      cpu:     500m
      memory:  500

## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?   `subscription`

Apply this config file.

In [27]:
!kubectl apply -f .\kubernetes\service.yaml

service/lb-subscription created


In [28]:
!kubectl get service

NAME              TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes        ClusterIP      10.96.0.1      <none>        443/TCP        126m
lb-subscription   LoadBalancer   10.96.202.56   <pending>     80:31647/TCP   27s


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

In [ ]:
!kubectl port-forward service/lb-subscription 9696:80

## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME               REFERENCE                 TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   1%/20%    1         3         1          27s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0% as there are no clients sending requests to the server.
> 
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release 
> from the `components.yaml` manifest:
> ```bash
> kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```

In [29]:
!kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3

horizontalpodautoscaler.autoscaling/subscription-hpa autoscaled


In [33]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         1          6m8s


## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing
`q6_test.py` script by putting the operator that sends the request to the subscription service into a loop.

```python
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.

## Question 8 (optional)

Run `kubectl get hpa subscription-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* 1    `<--`
* 2
* 3
* 4
